In [1]:
import numpy as np
import galFuncs
% matplotlib inline
from matplotlib import pyplot as plt
import time

In [2]:
Rshape = (2,2,2,1000,25,9)
recgShape = (2,2,1000,25,9)

In [3]:
center=0.0
std = 0.1
R = np.random.normal(center, std, size=np.product(Rshape)).reshape(Rshape)
rc = np.random.normal(center, std, size=np.product(recgShape)).reshape(recgShape)

In [4]:
print R.shape

(2, 2, 2, 1000, 25, 9)


In [5]:
def bootstrap(R, recg0, axisListRRand, axisListRMean, axisListRecg0Rand, axisListRecg0Mean, nBootstrap, noBootstrap=False):
    shape = np.array(R.shape)[axisListRRand]
    if noBootstrap:
        return np.array([np.nanmean(R, axis=tuple(axisListRMean))]), np.array([np.nanmean(recg0, axis=tuple(axisListRecg0Mean))])
    else:
        weights = galFuncs.getWeightArr(shape, nBootstrap)
        RBoot = galFuncs.bootstrapArr(R, axisListRRand, axisListRMean, weights)
        tmsBoot = galFuncs.bootstrapArr(recg0, axisListRecg0Rand, axisListRecg0Mean, weights)
        return RBoot, tmsBoot

axisListRRand, axisListRMean = [3,4], [2,3,4]
axisListRecg0Rand, axisListRecg0Mean = [2,3], [1,2,3]

nBootstrap=500
st = time.clock()

RBoot,tmsBoot = bootstrap(R, rc, axisListRRand, axisListRMean, axisListRecg0Rand, axisListRecg0Mean, nBootstrap, False)
RBoot, tmsBoot = RBoot.transpose(0,3,1,2), tmsBoot.transpose(0,2,1) # (nBootstrap, nrot, 2x2), (nBootstrap, nrot, g1_g2)
print time.clock() - st
rec_cosmoshear_boot = np.einsum('...jk,...k->...j', np.linalg.inv(RBoot), tmsBoot)
rec_cosmoshear, rec_cosmoshear_std = np.mean(rec_cosmoshear_boot, axis=0), np.std(rec_cosmoshear_boot, axis=0)
true_mean_shape, tmsstd = np.mean(tmsBoot, axis=0), np.std(tmsBoot, axis=0)

cshear_err = rec_cosmoshear_std 
tms_err = tmsstd 


25.31


* Have R (2x2, 0/90, ngal, nx^2, npixelsize)
* Get mean R from your bootstrapped sample -> (nBootstrap, npixelsize, 2x2)
* find recovered g
* Take mean and standard dev of recovered g's

In [6]:
print R.shape
print RBoot.shape
print tmsBoot.shape


(2, 2, 2, 1000, 25, 9)
(500, 9, 2, 2)
(500, 9, 2)


In [7]:
print rec_cosmoshear_boot[0,0]
print np.linalg.inv(RBoot[0,0]), tmsBoot[0,0]

[ 14.5653883  -27.84915153]
[[ 20586.06189041  -8693.95783328]
 [-39874.04826694  15397.53706764]] [ 0.00060118 -0.00025183]


In [8]:
print np.linalg.inv(RBoot[5,2])

[[-1129.42403644 -1078.99403121]
 [  -91.1082706   1285.93919527]]


In [9]:
# Each entry in RBoot is the average of 2*1000*25 entries

In [10]:
print RBoot.mean(), center
print RBoot.std(axis=0), std / np.sqrt(2*1000*25)


-4.791584400225978e-06 0.0
[[[0.00043778 0.00044641]
  [0.00044923 0.00043017]]

 [[0.00046394 0.00044101]
  [0.00043978 0.00042395]]

 [[0.00047126 0.00045331]
  [0.00046588 0.00045422]]

 [[0.00044002 0.0004578 ]
  [0.00044032 0.00045841]]

 [[0.00049361 0.00045725]
  [0.00042446 0.00044667]]

 [[0.00044621 0.00045169]
  [0.0004495  0.00045309]]

 [[0.00044588 0.00043339]
  [0.00042855 0.00045421]]

 [[0.00046085 0.00040935]
  [0.00046019 0.00046047]]

 [[0.00045022 0.00044602]
  [0.00044227 0.0004562 ]]] 0.00044721359549995795


## Unit test 2
### Negative sign gaussian ellipticity

In [11]:
rc[:,1,...] = -rc[:,0,...]
print rc.shape

(2, 2, 1000, 25, 9)


In [13]:
RBoot,tmsBoot = bootstrap(R, rc, axisListRRand, axisListRMean, axisListRecg0Rand, axisListRecg0Mean, nBootstrap, False)
RBoot, tmsBoot = RBoot.transpose(0,3,1,2), tmsBoot.transpose(0,2,1) # (nBootstrap, nrot, 2x2), (nBootstrap, nrot, g1_g2)
rec_cosmoshear_boot = np.einsum('...jk,...k->...j', np.linalg.inv(RBoot), tmsBoot)
rec_cosmoshear, rec_cosmoshear_std = np.mean(rec_cosmoshear_boot, axis=0), np.std(rec_cosmoshear_boot, axis=0)
true_mean_shape, tmsstd = np.mean(tmsBoot, axis=0), np.std(tmsBoot, axis=0)


rcshape: (x, rot, ngal, nx2, pixelsize)
bootstrap: pick random galaxies and subpixelshifts but *keep rots together* then average, leave x and ps out

[[-9.51310236e-16  7.84577624e-15]
 [-3.14909083e-15  3.67764175e-15]
 [-2.63946552e-16  1.01928228e-14]
 [-2.13945480e-13  3.70441859e-13]
 [-5.62508429e-15  1.66962339e-15]
 [-3.30091573e-15 -2.23076399e-15]
 [ 1.34341249e-15 -1.64712022e-16]
 [ 1.22606672e-14  1.22679822e-15]
 [ 8.02073458e-16  2.33982898e-15]] [[1.33418746e-13 1.13972194e-13]
 [1.16625551e-13 2.27715435e-13]
 [7.13627671e-14 2.02189828e-13]
 [4.90102595e-12 8.17350668e-12]
 [1.02015175e-13 3.77104813e-14]
 [7.71011793e-14 6.62472020e-14]
 [6.18933858e-14 5.02534951e-14]
 [2.35595051e-13 1.39890754e-13]
 [5.64909540e-14 4.08216308e-14]]
